### Work in progress

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import numpy as np
from os import listdir
import re

### Retrieve data files list

In [2]:
folder = './'
# get all file names
files = [f for f in listdir(folder) if f[-4:]=='.png']
# load one to get the dimensions
image = mpimg.imread(f'{folder}{files[0]}')
# get dimensions
shape = list(image.shape)
# remove alpha if necessary
if (shape[2]==4):
    shape[2] = 3
n = len(files)

### Read in data

In [4]:
y = np.zeros(shape=n,dtype=int)
X = np.zeros(shape=(n,*shape))
for i in range(len(files)):
    # get the hour from the file name
    y[i] = re.search('^([0-9]*)_',files[i])[1]
    # get the pixels, remove the alpha if needed and convert to 0-255
    X[i] = (mpimg.imread(f'{folder}{files[i]}')[:,:,:shape[2]] )
    
# transform y to label encoded
temp = y.copy()
y = np.zeros(shape=(n,12),dtype=int)
for i in range(n):
    y[i,temp[i]-1] = 1   

### Split data to training and test sets

In [5]:
# note, the input files are already shuffled
train = 0.8
cutoff = int(len(X) * train)

X_train = X[:cutoff]
y_train = y[:cutoff]

X_test = X[cutoff:]
y_test = y[cutoff:]

### Network

In [7]:
import theano
import theano.tensor as T

In [13]:
def new_weights(shape):
    return theano.shared(np.random.normal(scale=0.1,size=shape))

def new_bias(length):
    return theano.shared(np.zeros(length))

def new_fc_layer(X,num_inputs,num_outputs,use_relu=True,drop_out=None):
    weights = new_weights([num_inputs,num_outputs])
    bias = new_bias(num_outputs)
    layer = X.dot(weights) + bias
    if (use_relu):
        layer = T.nnet.relu(layer)
    if (drop_out is not None):
        ## TODO   
        pass
    return layer

def new_conv_layer(X,channels,k_size,n_kernels,use_pooling=True):
    #X : b_size,ch,rows,cols probably need to reshape X in the readin
    shape = [n_kernels,channels,k_size,k_size]
    kernels = new_weights(shape)
    bias = new_bias(n_kernels)
    
    layer = T.nnet.conv2d(input=X,filter=kernels)

    layer+= bias
    
    if use_pooling:
        layer = T.signal.pool.pool_2d()
    
    layer = tf.nn.relu(layer)
    
    return layer, kernels

def flatten(layer):
    shape = layer.shape
    num_features = np.array(shape[1:4],dtype=int).prod()
    layer_flat = tf.reshape(layer,[-1,num_features])
    return layer_flat,num_features

In [11]:
img_size = shape[0]
channels = shape[2]
img_size_flat = np.array(shape).prod()

x_image = tf.placeholder(tf.float32,shape=[None,img_size,img_size,channels],name='x_image')

y_true = tf.placeholder(tf.float32,shape=[None,12],name='y_true')

nk1 = 64
nk2 = 32

layer_conv1, conv1_weights = new_conv_layer(x_image,channels,3,nk1,False)  
layer_conv2, conv2_weights = new_conv_layer(layer_conv1,nk1,3,nk2,True)  
layer_flat,n_n = flatten(layer_conv2)

n_fc = 256

layer_fc1 = new_fc_layer(layer_flat,n_n,n_fc,True,0.5)

layer_fc2 = new_fc_layer(layer_fc1,n_fc,12,False)

y_pred = tf.nn.softmax(layer_fc2)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=layer_fc2,labels=y_true)

cost = tf.reduce_mean(cross_entropy)

optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

pred_labels = tf.math.round(y_pred)
correct_pred = tf.equal(pred_labels,y_true)
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [14]:
def optimize(num_iterations,X,y,batch_size):
    batches = int(len(X)/batch_size)
    for i in range(num_iterations):    
        for j in range(batches):
            x_batch = X[j*batch_size:(j+1)*batch_size]
            y_batch = y[j*batch_size:(j+1)*batch_size]
            feed_dict = {x_image: x_batch,y_true:y_batch}
            session.run(optimizer,feed_dict)
        cst,acc = session.run([cost,accuracy],feed_dict)
        print(f'Iteration {i}', f'cost: {cst}, accuracy: {acc}')